In [ ]:
import pandas as pd
import os

In [ ]:
caminho = os.path.join('..', '..', 'db_softtek') 
path_arq = os.path.abspath(caminho)

In [ ]:
db_orig = '\\FIAP_DataBase_v1.xlsx'  # Substitua pelo nome do seu arquivo


In [ ]:
# Carrega tabela de demandas:
tbl_demandas = pd.read_excel(path_arq+db_orig, 'DEMANDAS').drop_duplicates()

# Carrega tabela de Contratos:
tbl_contr = pd.read_excel(path_arq+db_orig, 'CONTRATOS').drop_duplicates()

# Carrega tabela de Custos:
tbl_custos = pd.read_excel(path_arq+db_orig, 'CUSTOS').drop_duplicates()
tbl_custos = tbl_custos.groupby(['EXERCICIO','PERIODO','CODIGO_AT']).agg({'VALOR_AT':'mean'}).reset_index()

# Carrega tabela de Equipe:
tbl_equipe = pd.read_excel(path_arq+db_orig, 'EQUIPE').drop_duplicates()


In [ ]:
#Demandas x Equipe:

tbl_1 = pd.merge(tbl_demandas, tbl_equipe, how ='left', on = 'IS').fillna(0).drop_duplicates()

# view1 x Custos:
tbl_2  = pd.merge(tbl_1, tbl_custos, how ='left', on = ['EXERCICIO','PERIODO','CODIGO_AT']).drop_duplicates()

In [ ]:
#Cruzamento por tbl_2 x contratos:

tbl_3  = pd.merge(tbl_2, tbl_contr, how ='left', left_on = ['PROJETO'],
                  right_on=['PROJETO'])


# Tirando duplicatas por contains de tipo chamado x tipo_demanda:

tbl_3['esta_contido'] = ''
for index, row in tbl_3.iterrows():
    if row['TIPO_CHAMADO'] in row['TIPO_DEMANDA']:
        tbl_3.at[index, 'esta_contido'] = 'Sim'

tbl_3 = tbl_3[tbl_3['esta_contido'] =='Sim'].reset_index().drop(columns=['index','esta_contido']).fillna(0)

tbl_3.describe()

In [ ]:
tbl_3.head()

In [ ]:
#Sainda primeira Tabela, analitica Full:

tbl_3.to_excel(path_arq+'\\cubo_anl\\tbl_full_analitica.xlsx',index=False)